**k-seq data analysis tutorial**

This is the Jupyter Notebook implementation of `k-seq` package with its core functions, including:

- Parse and overview count files
- Extract detected sequences
- [For this tutorial] Filter sequence before fitting
- Fit sequence into kinetic model

`k-seq` package [under development] provide core functions to parse, process, visualize and convert data from k-seq experiments. It is currently implemented as a `python` package for flexible and customized pipeline building. It will provide wrappers with standalone command line tools in future release.

# Environment setting

- Setup environmental variables `ENV`
- Add/reset the package directory (use package without `pip install k_seq`)
- Import core modules
  - `k_seq.data.pre_processing`: core module for data preprocessing before fitting
  - `k_seq.data.analysis`: module for preprocessing data analysis
  - `k_seq.fitting.fitting`: core module for kinetic model fitting
  - `k_seq.fitting.analysis`: module for fitting result analysis
  - `k_seq.utility`: module contains package-wide utility function
  - `util`: module contains global utility function

In [ ]:
# Set up environmental variables: change the directory to your system
class EnvVar:
    HOME_DIR = 'dir/to/home'
    PACKAGE_PATH = '/dir/to/k-seq/pkg'
    DATA_PATH= '/dir/to/count/file'
ENV = EnvVar()

# Add package directory for python kernel to import
import sys
if ENV.PACKAGE_PATH not in sys.path:
    sys.path = [ENV.PACKAGE_PATH] + sys.path

# import modules
import util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set up matplotlib dpi for visualization
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300

# Count file overview
Let's first look at the count files we will use for k-seq analysis

Jupyter Notebook can execute `shell` command within a `python` block using decorator '!'

E.g.

```Shell
!pwd
> current/direcotry/jupyter/notebook/kernel/starts
```

`python` variable's value can be called by `{variable}` in `shell` command

In [ ]:
# show current kernel directory


In [ ]:
# use shell command `ls` to list the file under ENV.DATA_PATH



In [ ]:
print(util.color.BOLD + util.color.BLUE + \
'Number of lines in file:' + util.color.END)
# use shell command `wc` to get the number of lines of R4A-inputA_S1_counts.txt



print(util.color.BOLD + util.color.BLUE + \
'Top 20 lines of file:' + util.color.END)
# use shell command `head` to check the top 20 lines of R4A-inputA_S1_counts.txt




## Import `k_seq` package

In [ ]:
import k_seq.data.pre_processing as pre_processing
import k_seq.data.analysis as analysis
import k_seq.fitting.fitting as fitting
import k_seq.fitting.analysis as fit_analysis
import k_seq.utility as utility

# Parse count files
## Use `pre_processing.SequencingSample` to parse single count file
Try to load in a single count file `R4A-1250A_S2_counts.txt`

In [ ]:
?pre_processing.SequencingSample()

In [ ]:
# Given file name R4A-1250A_S2_counts.txt, it follows pattern
#          [R4{exp_rep}-{byo, float}{seq_rep}_S{id,int}]_counts.txt

single_sample = pre_processing.SequencingSample(
    file_dirc= ENV.DATA_PATH + '/R4A-1250A_S2_counts.txt',
    name_pattern='[R4{exp_rep}-{byo, float}{seq_rep}_S{id,int}]_counts.txt',
    silent=False,
    x_value='byo'
)

**What contains in the a `SequencingSample` object?**

In [ ]:
# Use autocompletion and `?` to explore the function of `SequencingSample` object


**Use `survey_spike_in` to calculate the distribution of spike-in sequence in surrounding sequence space**

In [ ]:
results = single_sample.survey_spike_in(
    spike_in='AAAAACAAAAACAAAAACAAA',
    max_dist_to_survey=10,
    silent=False,
    inplace=False
)

**Use `matplotlib.pyplot` to visualize the local distribution of spike-in peak**

In [ ]:
# Visualize the peak of spike-in
import matplotlib.pyplot as plt

# Visualization code:












## Use `pre_processing.SequencingSampleSet` to parse a batch (folder) of count files 
### Parse count files from `~/data/count/`

In [ ]:
sample_set = pre_processing.SequencingSampleSet(

)

### explore the imported sample

In [ ]:
?sample_set

**Use `analysis.sequencing_sample_info_table` to print out an overview table**

In [438]:
?analysis.sequencing_sample_info_table()

**Use `analysis.sequencing_sample_info_plot` to plot an overview figure for samples**

In [ ]:
analysis.sequencing_sample_info_plot()

### Calculate quantification factor for each sample using spike in

$$
\text{Seq amount} = \frac{\text{Seq count}}{\text{spike-in count}}\times \text{spike-in amount} = \frac{\text{Seq count}}{\text{Total count}}\times \Large (\frac{\text{Total count}}{\text{spike-in count}}\times\text{spike-in amount}\Large )
$$

In [ ]:
# indicate spike-in amount to calculate quantification factors
spike_in_amounts = []
for i in range(4):
    spike_in_amounts += [4130, 1240, 826, 413, 207, 82.6, 41.3]

sample_set.get_quant_factors(

    
)

In [ ]:
sample_set.sample_set[0].spike_in

**Let's look at the overview again using `analysis.sequencing_sample_info_table/plot`**

In [ ]:
analysis.sequencing_sample_info_table(sample_set)
analysis.sequencing_sample_info_plot(sample_set)

### Filter sample
As an option, we can discard 0 concentraton samples as very limited sequencees from these samples

In [ ]:
?sample_set.filter_sample

In [ ]:
zero_sample_list = [sample.name for sample in sample_set.sample_set if sample.metadata['byo'] == 0]
print(zero_sample_list)
## How to get samples that are not 0?





In [ ]:
filtered_sample_set = sample_set.filter_sample(sample_to_keep=###TO FILL IN###,
                                               inplace=False)

In [ ]:
analysis.sequencing_sample_info_table(filtered_sample_set)
analysis.sequencing_sample_info_plot(filtered_sample_set)

# Extract valid sequences
## `SequencingSampleSet` $\to$ `SequenceSet`

In [ ]:
sequence_set = pre_processing.SequenceSet(sample_set=filtered_sample_set, remove_spike_in=True, note='JNN tutorial')

In [ ]:
?sequence_set

In [ ]:
sequence_set.dataset_info

In [ ]:
sequence_set.count_table

## How do these sequences distribute across samples

In [ ]:
sequence_set = analysis.survey_seqs_info(sequence_set)
count_bins, count_bins_weighted = analysis.survey_seq_occurrence(sequence_set=sequence_set, display=True)

## Normalize counts into reacted fraction

In [ ]:
zero_samples = [sample_name for sample_name in sequence_set.sample_info.keys() if '-0' in sample_name]
sequence_set.get_reacted_frac(
    input_average='median',
    black_list=zero_samples,
    inplace=True
)

In [ ]:
sequence_set.reacted_frac_table

## [Optional] Filter sequences

In [ ]:
?sequence_set.filter_seq

In [ ]:
# Find out a list of sequence to keep
seq_to_keep = sequence_set.reacted_frac_table.index[sequence_set.reacted_frac_table.isnull().sum(axis='columns') == 0]
seq_to_keep = list(filter(lambda x: len(x)==21, seq_to_keep))

In [ ]:
# Let's subsample sequences for this tutorial
seq_to_keep = np.random.choice(seq_to_keep, size=10)

In [ ]:
# apply filter
sequence_set_filtered = sequence_set.filter_seq(seq_to_keep=seq_to_keep, inplace=False)

In [ ]:
# check the new dataset
sequence_set_filtered.reacted_frac_table.shape

In [ ]:
# Only with known major sequences
major_seqs = [
    'ATTACCCTGGTCATCGAGTGA',
    'ATTACCCTGGTCATCGAGTGT',
    'CTACTTCAAACAATCGGTCTG',
    'CCACACTTCAAGCAATCGGTC',
    'CCGCTTCAAGCAATCGGTCGC',
    'CCGAGTTTCAAGCAATCGGTC',
    'AAGTTTGCTAATAGTCGCAAG'
]

# Filter sequences by major sequences



In [ ]:
# Save it on the disk for reference
util.dump_pickle(sequence_set_filtered, dirc='/mnt/storage/projects/k-seq/output/sequence_set_test.pkl',log='test sequenceset',overwrite=True)

# Fitting w/ uncertainty estimation
## Fit single function

In [ ]:
sequence_set_filtered = util.load_pickle('/mnt/storage/projects/k-seq/output/sequence_set_test.pkl')

In [ ]:
?fitting.fitting_single

In [ ]:
# Define the model to fit
def bfo_model(x, A, k):
    """
    Default kinetic model used in BFO k-seq fitting:
                    A * (1 - np.exp(-0.3371 * 90 * k * x))
    90: t, reaction time (min)
    0.3371: alpha, degradation adjustment parameter for BFO in 90 min

    :param x: predictor for the regression model, here is initial concentration of BFO
    :param A: parameter represents the maximal conversion of reactants
    :param k: parameter represents the apparent kinetic coefficient
    :return: reacted fraction given the independent variable x and parameter (A, k)
    """
    return A * (1 - np.exp(- 0.3371 * 90 * k * x *10e-6))

In [ ]:
for i in range(len(x_data)):
    print(x_data[i], '\t', y_data[i])

In [ ]:
fitting_res = fitting.fitting_single(x_data=x_data,
                                     y_data=y_data,
                                     func=bfo_model,
                                     missing_data_as_zero=False,
                                     bootstrap=True,
                                     bounds=((0, 0), (1, np.inf)),
                                     bs_depth=1000,
                                     bs_residue=False,
                                     bs_return_verbose=True,
                                     random_init=False)

In [ ]:
# check the fitting results
fitting_res[0]

In [ ]:
# visualize the fitting curve

fit_analysis.plot_fitting_single(seq_res=fitting_res, func=bfo_model,
                                 axis_labels=('[BFO]', 'Reacted fraction'),
                                 seq_name=seq_to_fit,
                                 legend_off=False,
                                 save_dirc=None)

## Fit all the sequence in `sequence_set`

In [ ]:
fitting.fitting_sequence_set(sequence_set=sequence_set_filtered, inplace=True, parallel_threads=6)